In [33]:
#Azure Openai嵌入模型部署不对大陆个人开放，我改用ollama的嵌入模型,向量数据库使用chromadb  2024_1_12

In [34]:
#langchain-chroma和chromadb兼容有问题，向量数据库改用Pinecone  2024_1_14

In [35]:
import ollama

In [ ]:
ollama.embeddings(model='nomic-embed-text', prompt='I told u')

In [39]:
#import chromadb.utils.embedding_functions as embedding_functions
from utils.OllamaEmbedding import OllamaEmbeddingFunction as oe

ollama_ef = oe(
    url="http://localhost:11434/api/embeddings",
    model_name="nomic-embed-text",
)

embeddings = ollama_ef(["This is my first text to embed",
                        "This is my second document"])


In [41]:

embeddings

[[0.1853868067264557,
  0.535433292388916,
  -3.27349591255188,
  -1.6316630840301514,
  0.6346610188484192,
  -0.7844298481941223,
  0.7711592316627502,
  0.4500104486942291,
  -0.3900703489780426,
  -1.1780610084533691,
  -1.1172101497650146,
  0.24217592179775238,
  1.389321208000183,
  1.2301981449127197,
  -1.1914476156234741,
  0.7381033301353455,
  1.4754787683486938,
  -1.370509386062622,
  -0.8170267343521118,
  1.4595359563827515,
  -0.7934385538101196,
  -0.23056373000144958,
  -1.0571552515029907,
  -0.032369956374168396,
  1.404036521911621,
  0.35039493441581726,
  -0.1630808264017105,
  0.3149219751358032,
  -1.5912001132965088,
  -0.3671587109565735,
  0.8121604323387146,
  -0.4767477512359619,
  -0.31510722637176514,
  -0.24938926100730896,
  0.2502032220363617,
  -0.5666965246200562,
  -0.6281144618988037,
  0.7519160509109497,
  0.13361969590187073,
  0.12159780412912369,
  0.5505712032318115,
  -0.5848843455314636,
  -0.019600078463554382,
  -0.3325035870075226,
  0

In [ ]:
import OllamaEmbeddingFunction
importlib.reload(OllamaEmbeddingFunction)

ModuleNotFoundError: No module named 'chromadb.telemetry.product.posthog'

In [25]:
#把ollamaembedding模型用于chroma数据库
from utils.OllamaEmbedding import OllamaEmbeddingFunction
#importlib.reload(chromadb)
#import chromadb.utils.embedding_functions as embedding_functions

ollama_ef = OllamaEmbeddingFunction(#embedding_functions.OllamaEmbeddingFunction(
    url="http://localhost:11434/api/embeddings",
    model_name="nomic-embed-text",
)

embeddings = ollama_ef(["This is my first text to embed",
                        "This is my second document"])


In [15]:
embeddings

[[0.1853868067264557,
  0.535433292388916,
  -3.27349591255188,
  -1.6316630840301514,
  0.6346610188484192,
  -0.7844298481941223,
  0.7711592316627502,
  0.4500104486942291,
  -0.3900703489780426,
  -1.1780610084533691,
  -1.1172101497650146,
  0.24217592179775238,
  1.389321208000183,
  1.2301981449127197,
  -1.1914476156234741,
  0.7381033301353455,
  1.4754787683486938,
  -1.370509386062622,
  -0.8170267343521118,
  1.4595359563827515,
  -0.7934385538101196,
  -0.23056373000144958,
  -1.0571552515029907,
  -0.032369956374168396,
  1.404036521911621,
  0.35039493441581726,
  -0.1630808264017105,
  0.3149219751358032,
  -1.5912001132965088,
  -0.3671587109565735,
  0.8121604323387146,
  -0.4767477512359619,
  -0.31510722637176514,
  -0.24938926100730896,
  0.2502032220363617,
  -0.5666965246200562,
  -0.6281144618988037,
  0.7519160509109497,
  0.13361969590187073,
  0.12159780412912369,
  0.5505712032318115,
  -0.5848843455314636,
  -0.019600078463554382,
  -0.3325035870075226,
  0

In [1]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

# if not os.getenv("PINECONE_API_KEY"):
#     os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

# pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key='pcsk_4f5KFA_9FP816yiirtNLHSwnuKPHGc6915XSNoxVuGpn4T75vA492YSYMymwVeofydS4sk')

In [8]:
import time

index_name = "langchain-test-index1"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [9]:
import langchain_ollama
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [10]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [11]:
#定义一批要处理的文档
from langchain_core.documents import Document
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [12]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['1b559dac-a48a-4fe8-b49c-60dca86505cc',
 '24de39bd-3585-481b-9af2-060916865cc6',
 'cd406136-63f7-4c5d-ae16-889594954849',
 '42197c35-5270-416d-b185-bf0a27403edd',
 'eb0c2b83-ded4-4f56-adc6-c5216f17af21']

In [25]:
results = vector_store.similarity_search(query="need plenty of",k=2)
for result in results:
    print(f"the {result.page_content:}------{result.metadata:}  \n")

the Goldfish are popular pets for beginners, requiring relatively simple care.------{'source': 'fish-pets-doc'}  

the Rabbits are social animals that need plenty of space to hop around.------{'source': 'mammal-pets-doc'}  

